In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import nolds

from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage


from scipy.signal import detrend
from tqdm import tqdm
from scipy.stats import linregress

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert, butter, filtfilt


In [ ]:
import my_functions2 as my_fun2

In [ ]:
subject = 1
runs = {
    "rest": [1],
    "motor_execution": [3, 7, 11],
    "motor_imagery": [4, 8, 12]
}

subject_data = load_and_preprocess_subject(subject, runs)

In [ ]:
%matplotlib qt

# Debug plot (rest condition)
quick_plot(subject_data["rest"], title=f"Subject {subject} | REST")

# Quick check of motor
quick_plot(subject_data["motor_execution"], title=f"Subject {subject} | MOTOR EXECUTION")

In [ ]:
# Extract epochs from each condition
epochs_rest = extract_clean_epochs(subject_data["rest"])
epochs_exec = extract_clean_epochs(subject_data["motor_execution"])
epochs_imag = extract_clean_epochs(subject_data["motor_imagery"])

In [ ]:
combined_rest = mne.concatenate_epochs([
    epochs_exec['rest'], 
    epochs_imag['rest'], 
    epochs_rest['rest']
])

combined_task = mne.concatenate_epochs([
    epochs_exec['task'], 
    epochs_imag['task']
])

# Shuffle task epochs 
shuffled_indices = np.random.permutation(len(combined_task))
combined_task = combined_task[shuffled_indices]

In [ ]:
%matplotlib qt 
#combined_task.plot(n_channels=8, title="Combined Task Epochs")

# DFA Analysis

α ≈ 0.7–1.0: Indicates long-range temporal correlations (LRTC) and near-critical dynamics

In [ ]:
alpha_rest = compute_dfa_from_epochs(combined_rest, picks=['C3', 'Cz', 'C4'])
alpha_task = compute_dfa_from_epochs(combined_task, picks=['C3', 'Cz', 'C4'])

print("Rest Alpha Values:", alpha_rest)
print("Task Alpha Values:", alpha_task)

In [ ]:
# Extract EEG signal (flatten Cz channel from rest condition)
signal = combined_task.get_data(picks="C3").reshape(-1)

# Compute DFA
alpha = compute_dfa(signal)
print(f"DFA exponent (Cz, rest): {alpha:.3f}")

In [ ]:
# Extract data (e.g., rest condition)
data_rest = combined_rest.get_data(picks=['C3', 'Cz', 'C4'])  # shape: (n_epochs, n_channels, n_times)
X_rest = data_rest.transpose(1, 0, 2).reshape(data_rest.shape[1], -1)  # (channels, time)

# Define segment sizes
segment_sizes = np.geomspace(30, X_rest.shape[1] // 5, 20).astype(int)

# Compute DFA
F_rest = compute_F(X_rest, segment_sizes)
alpha_rest, intercept_rest, _, _ = compute_DFA(segment_sizes, F_rest, fitting_range=(50, 10000))

# Plot
plot_dfa_fit(segment_sizes, F_rest, alpha_rest, intercept_rest, fitting_range=(50, 10000))

In [ ]:
# Extract data (e.g., rest condition)
data_rest = combined_task.get_data(picks=['C3', 'Cz', 'C4'])  # shape: (n_epochs, n_channels, n_times)
X_rest = data_rest.transpose(1, 0, 2).reshape(data_rest.shape[1], -1)  # (channels, time)

# Define segment sizes
segment_sizes = np.geomspace(30, X_rest.shape[1] // 5, 20).astype(int)

# Compute DFA
F_rest = compute_F(X_rest, segment_sizes)
alpha_rest, intercept_rest, _, _ = compute_DFA(segment_sizes, F_rest, fitting_range=(50, 10000))

# Plot
plot_dfa_fit(segment_sizes, F_rest, alpha_rest, intercept_rest, fitting_range=(50, 10000))

In [ ]:
alpha_beta_rest = run_band_limited_dfa(combined_rest, channel="Cz", band=(13, 30))
alpha_beta_task = run_band_limited_dfa(combined_task, channel="Cz", band=(13, 30))

print(f"Beta DFA (Rest - Cz): {alpha_beta_rest:.3f}")
print(f"Beta DFA (Task - Cz): {alpha_beta_task:.3f}")


In [ ]:
# 1. Get envelope from Cz, 13 Hz (alpha)
envelope = extract_wavelet_envelope(combined_rest, channel="Cz", sfreq=combined_rest.info['sfreq'], freq=13)

# 2. Define segment sizes
segment_sizes = np.unique(np.logspace(2, np.log10(len(envelope)/5), num=20, base=10).astype(int))

# 3. Compute F(n)
F = compute_fluctuation_function(envelope, segment_sizes)

# 4. Fit alpha
alpha, _ = compute_dfa_scaling(segment_sizes, F)

# 5. Plot
plt.figure(figsize=(8, 5))
plt.loglog(segment_sizes, F, 'o-', label=f"Cz (α = {alpha:.3f})")
plt.xlabel("log(Window size)")
plt.ylabel("log(Fluctuation)")
plt.title("DFA on Alpha Envelope – Cz (Rest)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()